In [ ]:
import numpy as np

from kafka_infra.MongoDbClient import MongoDbClient

client = MongoDbClient("svm_recognized_samples_signal")

In [ ]:
domain = np.arange(-1, 1, 0.02)

Create MongoDb query that returns details about model results

In [ ]:
def define_pipeline(x):
    pipeline = [
        {
            "$lookup": {
                "from": "generated_samples_signal",
                "localField": "sample_id",
                "foreignField": "sample_id",
                "as": "generated_sample_data"
            }
        },
        {
            "$unwind": "$generated_sample_data"
        },
        {
            "$project": {
                "sample_id": 1,
                "predicted_value": 1,
                "duration": 1,
                "normal_data": "$generated_sample_data.normal_data"
            }
        },
        {
            "$facet": {
                "normal_data_total": [
                    {
                        "$match": {
                            "normal_data": 1
                        }
                    },
                    {
                        "$count": "count"
                    }
                ],
                "normal_data_recognized": [
                    {
                        "$match": {
                            "normal_data": 1,
                            "predicted_value": { "$gte": x }
                        }
                    },
                    {
                        "$count": "count"
                    }
                ],
                "anomalous_data_total": [
                    {
                        "$match": {
                            "normal_data": 0
                        }
                    },
                    {
                        "$count": "count"
                    }
                ],
                "anomalous_data_recognized": [
                    {
                        "$match": {
                            "normal_data": 0,
                            "predicted_value": { "$lt": x }
                        }
                    },
                    {
                        "$count": "count"
                    }
                ]
            }
        },
        {
            "$project": {
                "normal_data_count": { "$arrayElemAt": ["$normal_data_total.count", 0] },
                "correct_recognized_normal_data": { "$arrayElemAt": ["$normal_data_recognized.count", 0] },
                "anomalous_data_count": { "$arrayElemAt": ["$anomalous_data_total.count", 0] },
                "correct_recognized_anomalous_data": { "$arrayElemAt": ["$anomalous_data_recognized.count", 0] }
            }
        }
    ]

    return pipeline


In [ ]:
sample_details = []
normal_data_count = -1
anomalous_data_count = -1

In [ ]:
for x in domain:
    pipeline = define_pipeline(x)
    result = client.aggregate(pipeline)
    for doc in result:
        if normal_data_count == -1:
            normal_data_count = doc['normal_data_count']
        if anomalous_data_count == -1:
            anomalous_data_count = doc['anomalous_data_count']
        try:
            doc['correct_recognized_normal_data']
        except KeyError:
            doc['correct_recognized_normal_data'] = 0
        try:
            doc['correct_recognized_anomalous_data']
        except KeyError:
            doc['correct_recognized_anomalous_data'] = 0
        sample_details.append({
            'factor': x,
            'correct_recognized_normal_data': doc['correct_recognized_normal_data'],
            'correct_recognized_anomalous_data': doc['correct_recognized_anomalous_data']
        })

In [ ]:
correct_recognized_normal_samples_ratio = []
correct_recognized_anomalous_samples_ratio = []

for sample in sample_details:
    correct_recognized_normal_samples_ratio.append(sample['correct_recognized_normal_data']/normal_data_count)
    correct_recognized_anomalous_samples_ratio.append(sample['correct_recognized_anomalous_data']/anomalous_data_count)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(domain, correct_recognized_normal_samples_ratio, label='Próbki normalne')
plt.plot(domain, correct_recognized_anomalous_samples_ratio, label='Anomalie')
plt.xlabel("Wartość podziału")
plt.ylabel("Procent poprawnie rozpoznanych próbek")
plt.ylim(-0.19, 1.05)
plt.legend(loc='lower left')
plt.show()

Find the value that model has the best accuracy

In [ ]:
min_difference = float('inf')
for i in range(0, len(correct_recognized_normal_samples_ratio)):
    difference = abs(correct_recognized_normal_samples_ratio[i] - correct_recognized_anomalous_samples_ratio[i])

    if difference < min_difference:
        closest_pair = (correct_recognized_normal_samples_ratio[i], correct_recognized_anomalous_samples_ratio[i])
        index = i
        min_difference = difference

print(index)
print(np.mean(closest_pair))

In [ ]:
print(sample_details[0])

In [ ]:
print(normal_data_count)
print(anomalous_data_count)

print(normal_data_count - 633)
print(anomalous_data_count - 135)

In [ ]:
durations = client.find({'duration': 1, '_id': 0})
values = []
for doc in durations:
    print(doc['duration'])
    values.append(doc['duration'])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

data = [np.random.randint(0, 141) for _ in range(1400)]

bins = list(range(0, 151, 10))

hist, edges = np.histogram(values, bins=bins)

plt.figure(figsize=(15, 6))
plt.bar(edges[:-1], hist, width=9)

plt.xlabel("Czas przetwarzania")
plt.ylabel("Liczba obserwacji")
plt.title('SVM')
plt.xticks(edges, labels=[f"{int(edge)}-{int(edge + 9)}" for edge in edges[:-1]])

plt.grid(axis='y')
plt.tight_layout()
plt.show()

In [ ]:
np.mean(values)